In [1]:
import os
import json
import numpy as np

from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.data.rmg import RMGDatabase
from rmgpy.species import Species
from rmgpy import settings

# Load the Mechanism Uncertainty Information

In [2]:
# Make sure you've loaded:
# RMG-Py/save_sensitivity (now you have to rebuild because you turned word-wrap off in chemkin)
# RMG-database/bm_fit_sensitivity

# Load an example ethane model generated using the trees
from rmgpy.tools.uncertainty import Uncertainty, process_local_results

chem = '/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_mech_main/chem_annotated.inp'
spec = '/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_mech_main/species_dictionary.txt'
uncertainty = Uncertainty(output_directory='./temp/uncertainty')
uncertainty.load_model(chem, spec)

In [3]:
# Load the database used to generate the model
uncertainty.load_database(
    thermo_libraries=['BurkeH2O2', 'CurranPentane','FFCM1(-)','primaryThermoLibrary', 'thermo_DFT_CCSDTF12_BAC', 'DFT_QCI_thermo', 'CBS_QB3_1dHR'],
    kinetics_families='default',
    reaction_libraries=['CurranPentane','FFCM1(-)','combustion_core/version5'],
    kinetics_depositories=['training'],
)


In [4]:
# Get the different reaction (and thermo- don't forget thermo) sources
uncertainty.extract_sources_from_model()
uncertainty.assign_parameter_uncertainties()

In [5]:
import json
def unpack_sensitivity(long_desc):
    start_str = 'sensitivities = '
    if start_str not in long_desc:
        return []
    start_index = long_desc.find(start_str) + len(start_str)
    sensitivities_str = long_desc[start_index:].replace("'", '"')
    sensitivities_str = sensitivities_str.replace("nan", '"-9999999"')
    return json.loads(sensitivities_str)

In [6]:
# List the reactions that came from nodes on the new trees and collect the families
print('These reactions were estimated using the new trees:')

families_to_lookup = set()

empty_derivatives = 0
for reaction_key in uncertainty.reaction_sources_dict.keys():
    if 'Rate Rules' in uncertainty.reaction_sources_dict[reaction_key].keys():
        family = reaction_key.family
        src = uncertainty.reaction_sources_dict[reaction_key]['Rate Rules'][1]
        if src['node'] != '':
            # look up the node name in the database
            if src['node'] not in uncertainty.database.kinetics.families[family].rules.entries.keys():
                print('NODE NOT FOUND\t', src['node'])
                continue
            
            entry = uncertainty.database.kinetics.families[family].rules.entries[src['node']][0]
            derivatives = unpack_sensitivity(entry.long_desc)
            if derivatives == []:
                # It's okay- I checked and I just haven't generated the derivatives for those families yet
                continue
            
            print(f'{reaction_key}\t({family}) \n\t{src["node"]}')
            print()
            
            # collect the set of families to lookup
            families_to_lookup.add(str(family))


print(families_to_lookup)


These reactions were estimated using the new trees:
C2H5(44) + C5H11-1(421) <=> nheptane(1)	(R_Recombination) 
	Root_N-1R->H_N-1CNOS->N_N-1COS->O_1CS->C_N-1C-inRing_Ext-2R-R_Ext-3R!H-R_N-Sp-3R!H=2R

NC3H7(93) + PC4H9(183) <=> nheptane(1)	(R_Recombination) 
	Root_N-1R->H_N-1CNOS->N_N-1COS->O_1CS->C_N-1C-inRing_Ext-2R-R_Ext-3R!H-R_N-Sp-3R!H=2R

H(14) + C[CH]CCCCC(682) <=> nheptane(1)	(R_Recombination) 
	Root_1R->H_N-2R->S_N-2CHNO->H_N-2CNO-inRing_Ext-2CNO-R_N-Sp-3R!H=2CCNNOO_N-2CNO->O_Ext-2CN-R

C2H5(44) + C[CH]CCCCC(682) <=> C2H4(11) + nheptane(1)	(Disproportionation) 
	Root_N-4R->H_4CNOS-u1_N-1R!H->O_N-4CNOS->O_Ext-4CNS-R_N-Sp-5R!H#4CCCNNNSSS_N-2R!H->S_N-5R!H->O_Sp-5CS-4CCNSS_Ext-4CNS-R

NC3H7(93) + C[CH]CCCCC(682) <=> C3H6(12) + nheptane(1)	(Disproportionation) 
	Root_Ext-1R!H-R_N-4R->O_N-Sp-5R!H=1R!H_Ext-4CHNS-R_N-6R!H->S_4CHNS->C_N-Sp-6BrBrBrCCCClClClFFFIIINNNOOOPPPSiSiSi#4C_6BrCClFINOPSi->C_N-1R!H-inRing_Ext-4C-R_2R!H->C

O2(2) + C[CH]CCCCC(682) <=> CCCCCC(C)O[O](763)	(R_Recombinat

In [7]:
print('# Node\tTraining Reaction\t\tdA_node/dA_train\tdE0_node/dA_train\tdn_node/dA_train')
for family in families_to_lookup:
    templateRxnMap = uncertainty.database.kinetics.families[family].get_reaction_matches(thermo_database=uncertainty.database.thermo, remove_degeneracy=True,
                                                                             get_reverse=True, exact_matches_only=False, fix_labels=True)
    
    for r, mechanism_reaction in enumerate(uncertainty.reaction_sources_dict.keys()):
        if mechanism_reaction.__class__.__name__ == 'PDepReaction':
            continue
        if mechanism_reaction.family != family:
            continue
            
        if 'Rate Rules' in uncertainty.reaction_sources_dict[mechanism_reaction].keys():
            # family = reaction_key.family
            src = uncertainty.reaction_sources_dict[mechanism_reaction]['Rate Rules'][1]
            node_name = src['node']
            if node_name != '':
                print(f'{mechanism_reaction.index}\t {mechanism_reaction} \n{node_name}')
                # look up the node name in the database
                if node_name not in uncertainty.database.kinetics.families[family].rules.entries.keys():
                    print('DERIVATIVE NOT YET COMPUTED\t',src['node'])
                    continue
            
                entry = uncertainty.database.kinetics.families[family].rules.entries[node_name][0]
                derivatives = unpack_sensitivity(entry.long_desc)
                
                for i, rxn in enumerate(templateRxnMap[node_name]):
                    N = 35
                    pad_length = N - len(str(rxn))
                    rxn_str = str(rxn) + " " * pad_length
                    #compare names from both sources to check order
                    #print("\n\t", derivatives[i]['name'],"\n\t", rxn_str)
                    dA = derivatives[i]['dA']
                    dE0 = derivatives[i]['dE0']
                    dn = derivatives[i]['dn']
                    if (dn == "-9999999"):
                        dn = np.nan
                    print('\t', rxn_str, "\t\t{0:.6f}".format(dA),
                          "\t\t{0:.6f}".format(dE0),
                          "\t\t{0:.6f}".format(dn))
                    print()
            else:
                print('No node for reaction ', mechanism_reaction) # this needs to complain more loudly
            
    
    
    

# Node	Training Reaction		dA_node/dA_train	dE0_node/dA_train	dn_node/dA_train
200	 C2H5(44) + C5H11-1(421) <=> nheptane(1) 
Root_N-1R->H_N-1CNOS->N_N-1COS->O_1CS->C_N-1C-inRing_Ext-2R-R_Ext-3R!H-R_N-Sp-3R!H=2R
	 C3H3 + C7H7 <=> C10H10              		0.089797 		0.000000 		-0.000040

	 C3H3 + C3H3 <=> C6H6-3              		0.089975 		0.000000 		0.000082

	 C3H5 + C3H5 <=> C6H10-2             		0.003108 		0.000000 		0.000167

	 C3H5 + C2H5 <=> C5H10-2             		0.003108 		0.000000 		0.000167

	 C3H5 + CH3 <=> C4H8-2               		0.003107 		0.000000 		0.000166

	 C11H23 + CH3 <=> C12H26             		0.249477 		0.000000 		-0.000083

	 C10H21 + C2H5 <=> C12H26-2          		0.003108 		0.000000 		0.000167

	 C9H19 + C3H7 <=> C12H26-3           		0.003105 		0.000000 		0.000165

	 C8H17 + C4H9-2 <=> C12H26-4         		0.003108 		0.000000 		0.000167

	 C7H15 + C5H11 <=> C12H26-5          		0.046068 		0.000000 		0.000154

	 C6H13 + C6H13 <=> C12H26-6          		0.003107 		0.000000 		0.0001

IndexError: list index out of range

# Save the Mechanism Uncertainty Information

In [104]:
import yaml
import cantera as ct
import pickle

In [41]:
# see if we can use the reaction as the dictionary key
gas1 = ct.Solution('/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_mech_main/nheptane.cti')
len(gas1.reactions())

/home/moon/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Converting `np.integer` or `np.signedinteger` to a dtype is deprecated. The current result is `np.dtype(np.int_)` which is not strictly correct. Note that the result depends on the system. To ensure stable results use may want to use `np.int64` or `np.int32`.
  


2266

In [136]:
# check for duplicate names - yes there are duplicate equations
reaction_names = []
reaction_IDs = []
for reaction_key in uncertainty.reaction_sources_dict.keys():
    try:
        out_reaction = reaction_key.to_cantera()
        if type(out_reaction) is list:
            reaction_names.append(out_reaction[0].equation)
            #for rxn in out_reaction:
            #    reaction_IDs.append(rxn.ID)
            reaction_IDs.append(out_reaction[0].ID)
        else:
            reaction_names.append(out_reaction.equation)
            reaction_IDs.append(out_reaction.ID)
    except AttributeError:
        print(f'Failed to convert {reaction_key}')
print('\nReaction equations')
print(len(reaction_names))
print(len(set(reaction_names)))
print('\nReaction IDs')
print(len(reaction_IDs))
print(len(set(reaction_IDs)))

Failed to convert O2(2) + H(14) <=> HO2(17)
Failed to convert H2O2(18) <=> OH(15) + OH(15)
Failed to convert H(14) + C2H2(26) <=> C2H3(23)
Failed to convert H(14) + C3H6(12) <=> NC3H7(93)

Reaction equations
2244
2226

Reaction IDs
2244
2244


In [44]:
class ReactionUncertainty():
    self.source = ''

In [67]:
print(len(uncertainty.kinetic_input_uncertainties))
print(len(uncertainty.reaction_sources_dict.keys()))

2248
2248


In [137]:
# Fill out uncertainty yaml
dict_file = {}
for i, reaction_key in enumerate(uncertainty.reaction_sources_dict.keys()):
    try:
        out_reaction = reaction_key.to_cantera()
        # convert all to list for identical handling of regular and multi-arrhenius reactions
        if type(out_reaction) is not list:
            out_reaction = [out_reaction]

        for rxn in out_reaction:
            rxn_uncertainty = {}
            
            # Note the source- assume only one source per reaction, or else this might overwrite existing reaction
            for src_key in uncertainty.reaction_sources_dict[reaction_key].keys():
                rxn_uncertainty['source'] = [src_key]
            dict_file[reaction_key] = [rxn_uncertainty]
            #dict_file['rxn'] = [rxn_uncertainty]
            #dict_file[rxn] = 0
        
        
    except AttributeError:
        print(f'Failed to convert {reaction_key} -- (falloff Troe)')
    if i>5:
        break
    i += 1
    rxn = out_reaction

print(f'{i} reactions')

6 reactions


In [138]:
output_path = '/home/moon/autoscience/uncertainty/partial_nheptane/nheptane_uncertainty.pickle'
print(dict_file)
with open(output_path, 'wb') as file:
    output = pickle.dump(dict_file, file)
    print(output)

{LibraryReaction(index=1, reactants=[Species(index=2, label="O2", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[3.53764,-0.00122828,5.36759e-06,-4.93128e-09,1.45955e-12,-1037.99,4.6718], Tmin=(100,'K'), Tmax=(1087.71,'K')), NASAPolynomial(coeffs=[3.16427,0.00169454,-8.00335e-07,1.5903e-10,-1.14891e-14,-1048.45,6.08303], Tmin=(1087.71,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), comment="""Thermo library: BurkeH2O2"""), molecule=[Molecule(smiles="[O][O]")], molecular_weight=(31.9988,'amu')), Species(index=14, label="H", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[2.5,-1.87769e-11,2.35957e-14,-9.62804e-18,1.20728e-21,25472.7,-0.459566], Tmin=(100,'K'), Tmax=(4762.74,'K')), NASAPolynomial(coeffs=[2.87627,-0.00031258,9.73655e-08,-1.34776e-11,6.99516e-16,25110.4,-2.86621], Tmin=(4762.74,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K'), comment="""Thermo library: BurkeH2O2"""), molecule=[Molecule(smiles="[H]")], molecular_weight=(1.00797,'amu'))], products=[Species

In [59]:
out_reaction[0] == out_reaction[1]

False

In [60]:
my_dict = {}
my_dict[out_reaction[0]] = 0
my_dict[out_reaction[1]] = 1
print(my_dict)

{<ElementaryReaction: H2 + O <=> H + OH>: 0, <ElementaryReaction: H2 + O <=> H + OH>: 1}


In [127]:
out_reaction[0].ID

'8'